<a href="https://colab.research.google.com/github/gigisy/Capstone-SP-IPT/blob/main/Cleaning_Dataset/CLEANING_DATASET__MahaData_(KELOMPOK_4)_DAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

# IMPORT DATASET

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Baca Dataset
path = '/content/drive/MyDrive/Data Proyek/Proyek Akhir/'

outgoing_df = pd.read_excel(path + "OUTGOING 7 BULAN.xlsx")
incoming_df = pd.read_excel(path + "combined_incoming.xlsx")
mvline_df = pd.read_excel(path + "mvline_gabungan.xlsx")
barang_out_df = pd.read_excel(path + "DATA BARANG OUT FROM ANY LOCATION (WMS).xlsx")
customer_df = pd.read_excel(path + "customers.xlsx")
supplier_df = pd.read_excel(path + "CUSTOMER SUPPLIER.xlsx")

# **CLEANING OUTGOING**

## Cek Tipe Data

In [ ]:
outgoing_df

In [ ]:
outgoing_df = outgoing_df.fillna(method='ffill')
display(outgoing_df)

In [ ]:
outgoing_df.info()

## Perbaikan Format Tipe

In [ ]:
outgoing_df['Done at'] = pd.to_datetime(outgoing_df['Done at'], format='%Y/%m/%d %H:%M:%S', errors='coerce')
outgoing_df['Done at'] = outgoing_df['Done at'].dt.date
display(outgoing_df)

In [ ]:
outgoing_df.info()

In [ ]:
outgoing_df['Sale Date'] = outgoing_df['Sale Date'].astype(str)
outgoing_df['Done at'] = outgoing_df['Done at'].astype(str)
outgoing_df.info()

In [ ]:
def convert_to_datetime(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y-%d-%m')  # 'Sale Date'
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%Y-%m-%d')  # 'Done at'
        except ValueError:
            try:
                return pd.to_datetime(date_str, format='%d-%m-%Y')  # Format untuk 'Done at' yang salah (DD-MM-YYYY) atau format lain yang mungkin
            except ValueError:
                return pd.NaT  # Jika semua format gagal, tetapkan sebagai NaT

outgoing_df['Sale Date'] = outgoing_df['Sale Date'].apply(convert_to_datetime)
outgoing_df['Done at'] = outgoing_df['Done at'].apply(convert_to_datetime)
print(outgoing_df.dtypes)

In [ ]:
outgoing_df['Sale Date'] = pd.to_datetime(outgoing_df['Sale Date'], format='%Y-%d-%m', errors='coerce')
outgoing_df['Done at'] = pd.to_datetime(outgoing_df['Done at'], format='%Y-%m-%d', errors='coerce')
# Menangani NaT dengan format yang sesuai

In [ ]:
def hitung_nilai_sebenarnya(nilai):
  """Menghitung nilai sebenarnya berdasarkan kondisi."""
  try:
    # Ganti titik dengan koma jika ada titik sebagai pemisah desimal
    nilai = str(nilai).replace('.', ',')
    parts = nilai.split(',')
    if len(parts) > 1 and parts[1] != '0':
      return float(nilai.replace(',', '.')) * 1000  # Kembalikan ke titik untuk perkalian
    else:
      return float(nilai.replace(',', '.'))  # Kembalikan ke titik
  except (ValueError, TypeError):
    return float(nilai)

outgoing_df['Inventory Moves/Internal Quantity'] = outgoing_df['Inventory Moves/Internal Quantity'].apply(hitung_nilai_sebenarnya)

In [ ]:
outgoing_df['Delay'] = outgoing_df['Done at'] - outgoing_df['Sale Date']

In [ ]:
outgoing_df

In [ ]:
outgoing_df.iloc[[5518]]

## Cleaning Data

In [ ]:
duplicate_rows = outgoing_df[outgoing_df.duplicated(keep='first')]
duplicate_rows

In [ ]:
all_duplicate_rows = outgoing_df[outgoing_df.duplicated(keep=False)]
all_duplicate_rows

In [ ]:
outgoing_df.drop_duplicates(keep='first', inplace=True)

In [ ]:
remaining_duplicates = outgoing_df[outgoing_df.duplicated(keep=False)]
remaining_duplicates

In [ ]:
empty_rows = outgoing_df[outgoing_df.isnull().any(axis=1)]
empty_rows

In [ ]:
zero_quantity_rows = outgoing_df[outgoing_df['Inventory Moves/Internal Quantity'] == 0]
zero_quantity_rows

In [ ]:
# Baris-baris yang ingin dihapus
index_to_drop = outgoing_df[outgoing_df['Inventory Moves/Internal Quantity'] == 0].index

# Hapus baris-baris
outgoing_df = outgoing_df.drop(index_to_drop)

In [ ]:
zero_quantity_rows = outgoing_df[outgoing_df['Inventory Moves/Internal Quantity'] == 0]
zero_quantity_rows

In [ ]:
outgoing_df

In [ ]:
# Display basic statistics for the 'outgoing_df' DataFrame.
outgoing_df.describe()

# **CLEANING INCOMING**

## Cek Tipe Data

In [ ]:
incoming_df

In [ ]:
incoming_df.info()

## Mengatasi Missing Value

In [ ]:
# Buat salinan dari DataFrame original
df_filled = incoming_df.copy()

# Isi NaN berdasarkan kondisi
last_valid_row = None

for i in range(len(incoming_df)):
    current_row = df_filled.iloc[i]

    # Jika baris saat ini memiliki Number dan Reference yang valid
    if not pd.isna(current_row['Number']) and not pd.isna(current_row['Reference']):
        last_valid_row = current_row

    # Jika baris saat ini memiliki NaN dan ada last_valid_row
    elif pd.isna(current_row['Number']) and last_valid_row is not None:
        # Isi dengan nilai dari last_valid_row
        df_filled.iloc[i, df_filled.columns.get_loc('Number')] = last_valid_row['Number']
        df_filled.iloc[i, df_filled.columns.get_loc('Reference')] = last_valid_row['Reference']

# Tampilkan hasil
print("\nHasil pengisian:")
display(df_filled)

In [ ]:
print(df_filled.iloc[17])

## Export Dataset

In [ ]:
df_filled.to_excel('incoming_BENERPLISS.xlsx', index=False)

## Merubah Tanggal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file = '/content/drive/MyDrive/Data Proyek/Proyek Akhir/incoming_BENERPLISS.xlsx'

In [ ]:
incoming_df1 = pd.read_excel(file)
incoming_df1

In [ ]:
incoming_df1['Planned Date']=pd.to_datetime(incoming_df1['Planned Date'], errors='coerce')

In [ ]:
incoming_df1['Planned Date']=incoming_df1['Planned Date'].dt.strftime('%Y-%m-%d')

In [ ]:
incoming_df1

In [ ]:
incoming_df1['Planned Date'].unique()

In [ ]:
incoming_df1.info()

In [ ]:
def convert_to_datetime(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y-%d-%m')  # 'Sale Date'
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%Y-%m-%d')  # 'Done at'
        except ValueError:
            try:
                return pd.to_datetime(date_str, format='%d-%m-%Y')  # Format untuk 'Done at' yang salah (DD-MM-YYYY) atau format lain yang mungkin
            except ValueError:
                return pd.NaT  # Jika semua format gagal, tetapkan sebagai NaT

incoming_df1['Planned Date'] = incoming_df1['Planned Date'].apply(convert_to_datetime)
incoming_df1.info()

In [ ]:
incoming_df1

## Mengubah Quantity

In [ ]:
def format_quantity(quantity):
    # Jika angka memiliki lebih dari satu koma, biarkan tanpa perubahan
    if quantity.count(',') > 1:
        return quantity

    # Jika angka tidak memiliki koma dan titik desimal, kembalikan nilai asli
    if ',' not in quantity and '.' not in quantity:
        return quantity

    # Jika ada koma di angka tetapi hanya satu, kita proses seperti biasa
    if ',' in quantity:
        # Hapus koma dan konversi ke float
        quantity = float(quantity.replace(',', ''))

        # Jika angka sudah memiliki tiga digit desimal atau lebih, biarkan seperti itu
        if '.' in str(quantity) and len(str(quantity).split('.')[1]) >= 3:
            return "{:,.3f}".format(quantity)  # Mempertahankan format dengan koma pemisah ribuan
        else:
            # Jika kurang dari tiga digit desimal, format menjadi tiga digit desimal
            return "{:,.3f}".format(quantity)
    else:
        # Jika tidak ada koma tetapi ada titik desimal
        if '.' in quantity:
            # Konversi ke float dan tambahkan tiga digit desimal
            return "{:.3f}".format(float(quantity))
        else:
            # Jika tidak ada koma dan tidak ada titik desimal, kembalikan nilai asli
            return quantity

# Terapkan fungsi ke kolom "Incoming Moves/Internal Quantity"
incoming_df1['Incoming Moves/Internal Quantity'] = incoming_df1['Incoming Moves/Internal Quantity'].astype(str).apply(format_quantity)

In [ ]:
incoming_df1['Incoming Moves/Internal Quantity'].unique()

## Mengatasi Duplikasi

In [ ]:
duplicate_rows = incoming_df1[incoming_df1.duplicated(keep='first')]
duplicate_rows

In [ ]:
all_duplicate_rows = incoming_df1[incoming_df1.duplicated(keep=False)]
all_duplicate_rows

In [ ]:
df_unique = incoming_df1.drop_duplicates(keep='first')
df_unique

In [ ]:
incoming_df.info()

## Mengubah Nama Produk

In [ ]:
import re

# ... (your existing code) ...

# Hapus semua simbol selain angka dan huruf pada kolom 'Inventory Moves/Product/Record Name'
incoming_df1['Incoming Moves/Product/Record Name'] = incoming_df1['Incoming Moves/Product/Record Name'].str.replace(r"[^a-zA-Z0-9 ]", "", regex=True)

# ... (rest of your code, including creating top_10 and plotting) ...

In [ ]:
incoming_df1

## Export Dataset Incoming yang sudah clean

In [ ]:
incoming_df1.to_excel('incoming_BISMILLAH.xlsx', index=False)

In [ ]:
from google.colab import files
files.download('incoming_BISMILLAH.xlsx')

In [ ]:
# Display basic statistics for the 'outgoing_df' DataFrame.
incoming_df1.describe()

# **CLEANING MVLINE**

## Cek Tipe Data

In [ ]:
mvline_df

In [ ]:
mvline_df.info()

## Cek Missing Value

In [ ]:
print(mvline_df.isnull().sum())

In [ ]:
# Menampilkan semua baris yang memiliki setidaknya satu missing value
baris_missing = mvline_df[mvline_df.isnull().any(axis=1)]
baris_missing

In [ ]:
# Hapus semua simbol selain angka dan huruf pada kolom 'Inventory Moves/Product/Record Name'
mvline_df['Product/Record Name'] = mvline_df['Product/Record Name'].str.replace(r"[^a-zA-Z0-9 ]", "", regex=True)

## Cek Duplikasi

In [ ]:
# Periksa duplikat di semua kolom
duplikat = mvline_df[mvline_df.duplicated(keep=False)]

# Tampilkan baris duplikat
display(duplikat)

# Hitung jumlah baris duplikat
jumlah_duplikat = duplikat.shape[0]
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

In [ ]:
all_duplicate_rows = mvline_df[mvline_df.duplicated(keep=False)]
all_duplicate_rows

In [ ]:
# hapus duplikat secara langsung di DataFrame asli (inplace)
mvline_df.drop_duplicates(keep='first', inplace=True)

In [ ]:
# Periksa duplikat di semua kolom
duplikat = mvline_df[mvline_df.duplicated(keep=False)]

# Tampilkan baris duplikat
display(duplikat)

# Hitung jumlah baris duplikat
jumlah_duplikat = duplikat.shape[0]
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

## Drop Quantity yang bernilai nol (0)

In [ ]:
# Filter baris dengan Quantity = 0
nol = mvline_df[mvline_df['Quantity'] == 0]

# Tampilkan baris dengan Quantity = 0
display(nol)

# Hitung jumlah baris dengan Quantity = 0
all = nol.shape[0]
print(f"Jumlah baris dengan Quantity 0 : {all}")

In [ ]:
# Dapatkan index baris-baris yang ingin dihapus
index_to_drop = mvline_df[mvline_df['Quantity'] == 0].index

# Hapus baris-baris tersebut
mvline_df = mvline_df.drop(index_to_drop)

In [ ]:
# Filter baris dengan Quantity = 0
nol = mvline_df[mvline_df['Quantity'] == 0]

# Tampilkan baris dengan Quantity = 0
display(nol)

# Hitung jumlah baris dengan Quantity = 0
all = nol.shape[0]
print(f"Jumlah baris dengan Quantity 0 : {all}")

In [ ]:
mvline_df.info()

In [ ]:
mvline_df

## Export Dataset yang sudah clean

In [ ]:
mvline_df.to_excel('mvline_CLEAN.xlsx', index=False)

In [ ]:
from google.colab import files
files.download('mvline_CLEAN.xlsx')

# **CLEANING DATA BARANG OUT FROM ANY LOC**

## Cek Tipe Data

In [ ]:
barang_out_df

In [ ]:
barang_out_df.info()

In [ ]:
def hitung_nilai_sebenarnya(nilai):
  """Menghitung nilai sebenarnya berdasarkan kondisi."""
  try:
    # Remove commas as thousands separators
    nilai = str(nilai).replace(',', '')
    # Replace dot with comma if present
    nilai = nilai.replace('.', ',')
    parts = nilai.split(',')
    if len(parts) > 1 and parts[1] != '0':
      return float(nilai.replace(',', '.')) * 1000  # Kembalikan ke titik untuk perkalian
    else:
      return float(nilai.replace(',', '.'))  # Kembalikan ke titik
  except (ValueError, TypeError):
    return float(nilai)  # If still fails, try directly as float

barang_out_df['Quantity'] = barang_out_df['Quantity'].apply(hitung_nilai_sebenarnya)

In [ ]:
barang_out_df.info()

In [ ]:
barang_out_df

## Merubah Format Nama Produk

In [ ]:
barang_out_df['Product/Record Name'] = barang_out_df['Product/Record Name'].str.replace(';', '').str.replace(' ', '')

In [ ]:
import re

barang_out_df['Product/Record Name'] = barang_out_df['Product/Record Name'].str.replace('[^a-zA-Z0-9]', '', regex=True)

## Cek Duplikasi

In [ ]:
# Periksa duplikat di semua kolom
duplikat = barang_out_df[barang_out_df.duplicated(keep=False)]

# Tampilkan baris duplikat
display(duplikat)

# Hitung jumlah baris duplikat
jumlah_duplikat = duplikat.shape[0]
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

In [ ]:
all_duplicate_rows = barang_out_df[barang_out_df.duplicated(keep=False)]
all_duplicate_rows

In [ ]:
# Hapus duplikat dan simpan ke DataFrame baru
df_tanpa_duplikat = barang_out_df.drop_duplicates(keep='first')

# Atau, hapus duplikat secara langsung di DataFrame asli (inplace)
barang_out_df.drop_duplicates(keep='first', inplace=True)

In [ ]:
# Periksa duplikat di semua kolom
duplikat = barang_out_df[barang_out_df.duplicated(keep=False)]

# Tampilkan baris duplikat
display(duplikat)

# Hitung jumlah baris duplikat
jumlah_duplikat = duplikat.shape[0]
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

## Drop Quantity yang Bernilai Nol (0)

In [ ]:
# Filter baris dengan Quantity = 0
nilai_nol = barang_out_df[barang_out_df['Quantity'] == 0]

# Tampilkan baris dengan Quantity = 0
display(nilai_nol)

# Hitung jumlah baris dengan Quantity = 0
jumlah_nol = nilai_nol.shape[0]
print(f"Jumlah baris dengan Quantity = 0: {jumlah_nol}")

In [ ]:
# Dapatkan index baris-baris yang ingin dihapus
index_to_drop = barang_out_df[barang_out_df['Quantity'] == 0].index

# Hapus baris-baris tersebut
barang_out_df = barang_out_df.drop(index_to_drop)

In [ ]:
# Filter baris dengan Quantity = 0
nilai_nol = barang_out_df[barang_out_df['Quantity'] == 0]

# Tampilkan baris dengan Quantity = 0
display(nilai_nol)

# Hitung jumlah baris dengan Quantity = 0
jumlah_nol = nilai_nol.shape[0]
print(f"Jumlah baris dengan Quantity = 0: {jumlah_nol}")

In [ ]:
barang_out_df

In [ ]:
barang_out_df.info()

## Export Dataset

In [ ]:
from google.colab import files

# Simpan DataFrame ke file Excel
barang_out_df.to_excel('barang out fix banget.xlsx', index=False)

# Unduh file
files.download('barang out fix banget.xlsx')

# **CLEANING JAN-AUG24 CUSTOMER SUPPLIER**

## **SHEETS CUSTOMER**

### Cek Tipe Data

In [ ]:
customer_df

In [ ]:
customer_df.info()

### Cek Missing Value

In [ ]:
print(customer_df.isnull().sum())

In [ ]:
# Menampilkan semua baris yang memiliki setidaknya satu missing value
baris_missing = customer_df[customer_df.isnull().any(axis=1)]
print(baris_missing)

### Mengubah Nama Produk

In [ ]:
# Hapus semua simbol selain angka dan huruf pada kolom PRODUCTID
customer_df['PRODUCTID'] = customer_df['PRODUCTID'].str.replace(r"[^a-zA-Z0-9 ]", "", regex=True)

### Cek Duplikasi

In [ ]:
# Periksa duplikat di semua kolom
duplikat = customer_df[customer_df.duplicated(keep=False)]

# Tampilkan baris duplikat
display(duplikat)

# Hitung jumlah baris duplikat
jumlah_duplikat = duplikat.shape[0]
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

### Drop QUANTITY = 0

In [ ]:
# Filter baris dengan Quantity = 0
nol = customer_df[customer_df['QTY'] == 0]

# Tampilkan baris dengan Quantity = 0
display(nol)

# Hitung jumlah baris dengan Quantity = 0
all = nol.shape[0]
print(f"Jumlah baris dengan Quantity 0 : {all}")

In [ ]:
customer_df

In [ ]:
customer_df.info()

### Export Dataset

In [ ]:
customer_df.to_excel('JAN-AUG24 CUSTOMER.xlsx', index=False)

In [ ]:
from google.colab import files
files.download('JAN-AUG24 CUSTOMER.xlsx')

## **SHEETS SUPPLIER**

### Cek Tipe Data

In [ ]:
supplier_df

In [ ]:
supplier_df.info()

### Cek Duplikasi

In [ ]:
# Periksa duplikat di semua kolom
duplikat = supplier_df[supplier_df.duplicated(keep=False)]

# Tampilkan baris duplikat
display(duplikat)

# Hitung jumlah baris duplikat
jumlah_duplikat = duplikat.shape[0]
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

### Mengubah Nama Produk

In [ ]:
# Hapus semua simbol selain angka dan huruf pada kolom PRODUCTID
supplier_df['PRODUCTID'] = supplier_df['PRODUCTID'].str.replace(r"[^a-zA-Z0-9 ]", "", regex=True)

### Cek Missing Value

In [ ]:
print(supplier_df.isnull().sum())

In [ ]:
# Menampilkan semua baris yang memiliki setidaknya satu missing value
baris_missing = supplier_df[supplier_df.isnull().any(axis=1)]
print(baris_missing)

### Drop QUANTITY = 0

In [ ]:
# Filter baris dengan Quantity = 0
nol = supplier_df[supplier_df['QTY'] == 0]

# Tampilkan baris dengan Quantity = 0
display(nol)

# Hitung jumlah baris dengan Quantity = 0
all = nol.shape[0]
print(f"Jumlah baris dengan Quantity 0 : {all}")

In [ ]:
supplier_df

In [ ]:
supplier_df.info()

### Export Dataset

In [ ]:
supplier_df.to_excel('supplier.xlsx', index=False)

In [ ]:
from google.colab import files
files.download('supplier.xlsx')